# Priority Queues And Heaps

## Priority Queue ADT

A priority queue stores elements with an associated priority. Elements with the highest priority are the first ones out.

- `insertItem(k, e)`: Inserts element `e` with key `k`
- `maxElement()`: Returns an element with mazimum priority, error if queue is empty
- `removeMax()`: Remove and return the highest priority element, error if queue is empty
- `isEmpty()`: Returns true if the queue is empty, false otherwise

## Search tree implementation

A straitforward implementation would use binary search trees were elements would be stored by priority. This would have a runtime of $\Theta(h)$ (where $h$ is the height of the tree), except `isEmpty()` which is clearly $\Theta(1)$. Using AVL trees we can reduce this bound to $\Theta(\lg n)$.

## Abstract Heaps

For a binary tree, the $i$th _level_ is all vertices distance $i$ from the root. For example, the $0$th level is the root, the $1$st level is the children of the root, etc. Recall that for an almost complete binary tree all levels except the last will have the maximum vertices (that is the $i$th level has $2^i$ vertices).

![](res/complete_trees.png)

The left most is the only "almost complete" binary tree.

An almost complete binary tree with $n$ internal vertices has height $\lg \lfloor \lg n \rfloor + 1$.

First recall the a binary tree of height $h$ has a maximum of $2^{h+1}-1$ vertices ($2^h$ leaves and $2^h - 1$ internal vertices).

Let $n$ denote the number of vertices of an almost complete tree of height $h$.

$n$ is greater than the number of internal veritces of a complete tree of height $h-1$. That is $2^{h-1} - 1 < n$ or $2^{h-1} \leq n$. $n$ is atmost the number of vertices of a tree of height $h$, thus the bounds for $n$ are:

$$
2^{h-1} \leq n \leq 2^h - 1
$$

For all $n$ where $2^{h-1} \leq n \leq 2^h - 1$ we get $\lfloor \lg n \rfloor = h - 1$

A _heap_ is an almost complete binary tree with the condition that every vertex $v$ (except the root), the key stored at $v$ is less than the key stored at its parent.

### Insertion

Create a new last vertex and insert. If the value is larger than the parent, swap them until the tree is a heap again.

```
Algorithm insertItem(k, e)
    create new last vertex v
    while v is not the root and k > v.parent.key do
        store the item stored at v.parent at v
        v <- v.parent
    store (k, e) at v
```

The loop runs for atmost $h$, thus bt the theorem above the runtime os $\Theta(h) = \Theta(\lg n)$

### Max Element

By the defintion of the heap the max element will be store in the root. Thus the `maxElement` operation will be $\Theta(1)$.

### Removing Max

We cant easily remove the root of the tree, thus we swap the root with the last vertex, remove the last vertex then repair the tree.

- Starting at the root $v$, if $v$ is the maximum of $v$ and $v$'s children, then the tree is a heap since the subtrees rooted at the children are already heaps.
- If $v$ less than one of its children, then $v$ is swapped with the child and we apply heapify recursivly

```
Algorithm heapify(v)
    if v.left is an internal vertex and v.left.key > v.key then
        s <- v.left
    else
        s <- v
    if v.right is an internal vertex and v.right.key > s.key then
        s <- v.right
    if s != v then
        exchange the items of v and s
        heapify(s)
```

The runtime of `heapify` can be expressed in terms of $h$:

$$
T_{heapify}(h) = \Theta(1) + T_{heapify}(h-1)
$$

To solve this recurrence we will first show that for $f: \mathbb{N} \rightarrow \mathbb{R}$ such that for all $n \geq 1$

$$
f(n) = \Theta(1) + f(n-1)
$$

Then $f(n)$ is $O(n)$.

First let $n_0 \in \mathbb{N}$ and $c > 0$ such that $f(n) \leq c + f(n-1)$ for all $n \geq n_0$. Let $d = f(n_0)$, we claim that:

$$
f(n) \leq d + c(n - n_0)
$$

for all $n \geq n_0$. We shall prove this by induction, suppose $n > n_0$ and $f(n) \leq d + c(n - n_0)$ holds for $n-1$ then:

$$
f(n) \leq c + f(n-1) \leq c + d + c(n - 1 - n_0) = d + c(n - n_0)
$$

Finally $d + c(n - n_0) \leq d + cn = O(n)$. By a simular argument $f = \Omega(n)$, thus $f = \Theta(n)$

Appling this to $T_{heapify}$ we get $T_{heapify} = \Theta(h) = \Theta(\log n)$

## Array Based Implementation

The reason for heaps is that they can be stored in arrays in an efficent mannor. Each level is stored conseqtivley, thus we get:

| index | node |
| :--- | :--- |
| 0 | root |
| 1 | left child of root |
| 2 | right child of root |
| ... | ... |

This representation allows us to navigate the tree with simple arithmetic, for a node $v$:

| target | operation |
| --- | --- |
| left child of $v$ | $2v + 1$ | 
| right child of $v$ | $2v + 2$ |
| parent of $v$ | $\left\lfloor \frac{n-2}{2} \right\rfloor$ |

We only ever insert or remove from the end of the array, thus a dynamic array implementation (which has a $\Theta(1)$ amortised insert/remove time) will have the following runtimes:

| method | running time |
| :--- | :--- |
| `insertItem` | $\Theta(\log n)$ (amortised) |
| `maxElement` | $\Theta(1)$ |
| `insertItem` | $\Theta(\log n)$ (amortised) |
| `insertItem` | $\Theta(1)$ |

## Build Heap

In many applications it is usefull to take an array of items and turn it into a heap.

```
Algorithm buildHeap(H)
    n <- H.length
    for v <- floor((n-2)/2) downto 0 do
        heapify(v)
```

Subtrees of height 1 are by definition already heaps, thus we start at $\left\lfloor \frac{n-2}{2} \right\rfloor$, the maximum $v$ such that $2v - 1 \leq n-1$ i.e. the last vertex with atleast one child. Then we apply heapify from the bottom up, transforming subtrees into heaps. One would assume that for $n$ items `buildHeap` would take $O(n \log n)$ since heapify is $O(\log n)$ however we can do better.

Let $h(v)$ denote the height of vertex $v$ and $m = \left\lfloor \frac{n-2}{2} \right\rfloor$.

$$
\begin{align}
T_{buildHeap}(n) &= \sum_{v=0}^m \Theta(1) + T_{heapify}(h(v)) \\
&= \Theta(m) + \sum_{v=0}^m \Theta(h(v)) \\
&= \Theta(m) + \Theta\left(\sum_{v=0}^m h(v)\right)
\end{align}
$$

For $\sum_{v=0}^m h(v)$ we observe that

$$
\sum_{v=0}^m h(v) = \sum_{i=1}^h i \cdot (\text{number of vertices of height $i$})
$$

That is the sum of the heights of vertices 0 to $m$ is equivalent to the level $i$ times the vertices at that level.

$$
\begin{align}
\sum_{i=1}^h i \cdot (\text{number of vertices of height $i$}) &= 
\sum_{i=1}^{h-1} (h-i) \cdot (\text{number of vertices on level $i$}) \\
&\leq \sum_{i=0}^{h-1} (h-i) \cdot 2^i
\end{align}
$$

Using the fact that

$$
\sum_{i=0}^\infty \frac{i+1}{2^i} = 4
$$

Then

$$
\begin{align}
\sum_{i=0}^{h-1} (h-i) \cdot 2^i &= n \frac{\sum_{i=0}^{h-1} (h-i)\cdot 2^i}{n} \\
&= n \sum_{i=0}^{h-1} (h-i) \cdot \frac{2^i}{n} \\
&\leq n \sum_{i=0}^{h-1} (h-i) \cdot 2^{i-(h-1)} & \text{since $n \geq 2^{h-1}$} \\
&= n \sum_{i=0}^{h-1} (i+1) \cdot 2^{-i} \\
&\leq n \sum_{i=0}^\infty \frac{i+1}{2^i} \\
&= 4n
\end{align}
$$

And thus $T_{buildHeap}(n) = \Theta(m) + \Theta(n) = \Theta(n)$